## Train a model locally or remote

In [1]:
from sklearn import datasets, svm
digits = datasets.load_digits()
clf = svm.SVC(gamma=0.001, C=100.,probability=True)
clf.fit(digits.data[:-1], digits.target[:-1])
clf.predict(digits.data[-1:])

array([8])

In [2]:
!pip show scikit-learn

Name: scikit-learn
Version: 0.22.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages
Requires: joblib, scipy, numpy
Required-by: 


### Save model file

In [3]:
!pip install joblib
from joblib import dump
dump(clf, 'model.joblib')

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


['model.joblib']

## Step 1 : Write a model transform script

#### Make sure you have a ...

- "load_model" function
    - input args are model path
    - returns loaded model object
    - model name is the same as what you saved the model file as (see above step)
<br><br>
- "predict" function
    - input args are the loaded model object and a payload
    - returns the result of model.predict
    - make sure you format it as a single (or multiple) string return inside a list for real time (for mini batch)
    - from a client, a list  or string or np.array that is sent for prediction is interpreted as bytes. Do what you have to for converting back to list or string or np.array
    - return the error for debugging


In [4]:
%%writefile modelscript_sklearn.py
import sklearn
from joblib import load
import numpy as np
import os

#Return loaded model
def load_model(modelpath):
    print(modelpath)
    clf = load(os.path.join(modelpath,'model.joblib'))
    print("loaded")
    return clf

# return prediction based on loaded model (from the step above) and an input payload
def predict(model, payload):
    try:
        # locally, payload may come in as an np.ndarray
        if type(payload)==np.ndarray:
            out = [str(model.predict(np.frombuffer(payload).reshape((1,64))))]
        # in remote / container based deployment, payload comes in as a stream of bytes
        else:
            out = [str(model.predict(np.frombuffer(payload[0]['body']).reshape((1,64))))]
    except Exception as e:
        out = [type(payload),str(e)] #useful for debugging!
    
    return out

Overwriting modelscript_sklearn.py


## Does this work locally? (not "_in a container locally_", but _actually_ in local)

In [5]:
from modelscript_sklearn import *
model = load_model('.')

.
loaded


In [6]:
predict(model,digits.data[-1:])[0]

'[8]'

### ok great! Now let's install ezsmdeploy
In some cases, installs fail due to an existing package installed called greenlet.
This is not a direct dependency of ezsmdeploy but interferes with the installation. 
To fix this, either install in a virtualenv as seen above, or do:
pip install ezsmdeploy[locust] --ignore-installed greenlet

In [7]:
!ls

build		     model.joblib
dist		     modelscript_sklearn.py
downloads	     notebooks
env		     __pycache__
extractedmodel	     README.rst
ezsmdeploy	     setup.py
ezsmdeploy.egg-info  src
model1.tar.gz	     Using ezsmdeploy for sklearn deployments (1).ipynb
model2.tar.gz


In [8]:
!pip uninstall ezsmdeploy -y

Found existing installation: ezsmdeploy 1.0.0
Uninstalling ezsmdeploy-1.0.0:
  Successfully uninstalled ezsmdeploy-1.0.0


In [9]:
!pip install -e .

Obtaining file:///home/ec2-user/SageMaker/easy-amazon-sagemaker-deployments
  Running setup.py develop for ezsmdeploy
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [10]:
import ezsmdeploy

#### If you have been running other inference containers in local mode, stop existing containers to avoid conflict

In [11]:
!docker container stop $(docker container ls -aq) >/dev/null

"docker container stop" requires at least 1 argument.
See 'docker container stop --help'.

Usage:  docker container stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers


## Deploy locally

In [12]:
!rm -r src -f

In [ ]:
import subprocess
cmd = "chmod +x src/build-docker.sh  & sudo ./src/build-docker.sh {}"
        
with open('src/dockeroutput.txt', 'w') as f:
    p = subprocess.Popen(cmd.format("name"), stdout=f, shell=True)

In [ ]:
!cat src/dockeroutput.txt

In [13]:
ez = ezsmdeploy.Deploy(model = ['model.joblib'], # if you intend to add models later, pass model as list, otherwise str
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.22.1','numpy','joblib'], #or pass in the path to requirements.txt
                  instance_type = 'local',
                  autoscale = True,
                  #framework = 'sklearn', # not required if you provide requirements
                  wait = True)

0:00:00.214773 | compressed model(s)
0:00:00.326441 | uploaded model tarball(s) ; check returned modelpath
0:00:00.327276 | added requirements file
0:00:00.333171 | added source file
0:00:00.334490 | added Dockerfile
0:00:00.336207 | added model_handler and docker utils
0:00:00.336305 | building docker container
Start process
process running in background
0:01:11.912387 | built docker container
2m∙∙● 

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


0:01:14.316513 | created model(s). Now deploying on local
∙●∙ Attaching to tmp1qpaqven_algo-1-8x2kh_1
algo-1-8x2kh_1  | 2020-10-31 12:30:10,704 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
algo-1-8x2kh_1  | MMS Home: /usr/local/lib/python3.5/dist-packages
algo-1-8x2kh_1  | Current directory: /
algo-1-8x2kh_1  | Temp directory: /tmp
algo-1-8x2kh_1  | Number of GPUs: 0
algo-1-8x2kh_1  | Number of CPUs: 2
algo-1-8x2kh_1  | Max heap size: 864 M
algo-1-8x2kh_1  | Python executable: /usr/bin/python3
algo-1-8x2kh_1  | Config file: /etc/sagemaker-mms.properties
algo-1-8x2kh_1  | Inference address: http://0.0.0.0:8080
algo-1-8x2kh_1  | Management address: http://0.0.0.0:8080
algo-1-8x2kh_1  | Model Store: /.sagemaker/mms/models
algo-1-8x2kh_1  | Initial Models: ALL
algo-1-8x2kh_1  | Log dir: /logs
algo-1-8x2kh_1  | Metrics dir: /logs
algo-1-8x2kh_1  | Netty threads: 0
algo-1-8x2kh_1  | Netty client threads: 0
algo-1-8x2kh_1  | Default workers per model: 2
algo-1-8x2kh_1  | Blacklist Regex: 

## Test containerized version locally

In [ ]:
out = ez.predictor.predict(digits.data[-1:].tobytes()).decode()
out

In [ ]:
!docker container stop $(docker container ls -aq) >/dev/null

## Deploy on SageMaker

In [ ]:
ezonsm = ezsmdeploy.Deploy(model = ['model.joblib','model.joblib'], # example of multimodel endpoint. 
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.22.1','numpy','joblib'])

In [ ]:
#!./src/build-docker.sh test

In [ ]:
out = ezonsm.predictor.predict(digits.data[-1:].tobytes(),target_model='model1.tar.gz').decode() 
out

### Install the additional locust testing functionality to enable automated testing

In [ ]:
!pip install ezsmdeploy[locust]

In [ ]:
ezonsm.test(input_data=digits.data[-1:].tobytes(), target_model='model1.tar.gz',usercount=20,hatchrate=10,timeoutsecs=10)

In [ ]:
import pandas as pd

pd.read_csv('src/locuststats_stats.csv')

In [ ]:
ezonsm.predictor.delete_endpoint()